In [1]:
# ===============================================
# 🧠 TinyLlama + CSV + Voice + PDF Chat System
# ===============================================

# --- Step 1: Install dependencies ---
!pip install fastapi uvicorn reportlab pandas requests SpeechRecognition pyttsx3 pyaudio

  Using cached speechrecognition-3.14.3-py3-none-any.whl.metadata (30 kB)
  Using cached pyttsx3-2.99-py3-none-any.whl.metadata (6.2 kB)
  Using cached PyAudio-0.2.14-cp312-cp312-win_amd64.whl.metadata (2.7 kB)
  Using cached comtypes-1.4.13-py3-none-any.whl.metadata (7.2 kB)
  Using cached pypiwin32-223-py3-none-any.whl.metadata (236 bytes)
Using cached speechrecognition-3.14.3-py3-none-any.whl (32.9 MB)

   -------- ------------------------------- 1/5 [SpeechRecognition]
   -------- ------------------------------- 1/5 [SpeechRecognition]
   -------- ------------------------------- 1/5 [SpeechRecognition]
   -------- ------------------------------- 1/5 [SpeechRecognition]
   -------- ------------------------------- 1/5 [SpeechRecognition]
   -------- ------------------------------- 1/5 [SpeechRecognition]
   -------- ------------------------------- 1/5 [SpeechRecognition]
   -------- ------------------------------- 1/5 [SpeechRecognition]
   -------- ------------------------------- 1/

In [2]:
# --- Step 2: Import libraries ---
import pandas as pd
import requests
import speech_recognition as sr
import pyttsx3
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas


In [3]:
# --- Step 3: Load CSV Data ---
csv_path = r"C:\Users\Aadhya\Downloads\TinyLlamaVoiceChat\voice.csv"
data = pd.read_csv(csv_path)
print("✅ CSV Loaded Successfully!")
display(data.head())

✅ CSV Loaded Successfully!


,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [4]:
# --- Step 4: Voice input setup ---
recognizer = sr.Recognizer()
engine = pyttsx3.init()

def listen_to_user():
    """Capture user voice input and convert to text."""
    with sr.Microphone() as source:
        print("\n🎤 Speak now...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print("🗣️ You said:", text)
            return text
        except sr.UnknownValueError:
            print("❌ Sorry, I didn't catch that.")
            return None
        except sr.RequestError:
            print("⚠️ Speech recognition service unavailable.")
            return None

def speak_text(text):
    """Convert text to speech."""
    engine.say(text)
    engine.runAndWait()

In [5]:
# --- Step 5: Query TinyLlama via LM Studio ---
def query_tinyllama(user_text, context=""):
    """Send prompt to TinyLlama model running on LM Studio."""
    payload = {
        "model": "tinyllama",
        "messages": [
            {"role": "system", "content": f"You are a helpful assistant. Use this CSV data for context:\n{context}"},
            {"role": "user", "content": user_text}
        ]
    }
    try:
        response = requests.post("http://127.0.0.1:1234", json=payload)
        response_json = response.json()
        answer = response_json["choices"][0]["message"]["content"]
        return answer
    except Exception as e:
        return f"⚠️ Error contacting TinyLlama: {e}"

In [6]:

# --- Step 6: Prepare CSV context (sample 5 rows as summary) ---
context = data.head(5).to_string()


In [7]:
# --- Step 7: Chat interaction loop ---
chat_history = ""

while True:
    print("\n🎧 Listening for your question (say 'stop' to end)...")
    user_input = listen_to_user()
    if not user_input:
        continue
    if "stop" in user_input.lower():
        print("👋 Ending conversation.")
        break

    # Query TinyLlama
    response = query_tinyllama(user_input, context)
    print("\n🤖 TinyLlama:", response)
    speak_text(response)

    # Append to chat history
    chat_history += f"You: {user_input}\nTinyLlama: {response}\n\n"


🎧 Listening for your question (say 'stop' to end)...

🎤 Speak now...


KeyboardInterrupt: 

In [8]:
# --- Step 8: Save conversation as PDF ---
pdf_filename = "TinyLlama_Chat.pdf"
c = canvas.Canvas(pdf_filename, pagesize=A4)
width, height = A4
y = height - 50

for line in chat_history.split("\n"):
    c.drawString(50, y, line)
    y -= 20
    if y < 50:
        c.showPage()
        y = height - 50

c.save()
print(f"\n✅ Chat saved as PDF: {pdf_filename}")


✅ Chat saved as PDF: TinyLlama_Chat.pdf
